<a href="https://colab.research.google.com/github/npd00/mtg_data_enrichment/blob/main/ManaBoxDataExpansion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before running the script please upload your manabox collection to the default folder uner files with the file name "ManaBox_Collection.xsv"

In [ ]:
#Imports
import pandas as pd
import requests
import json
import gzip
import io
import time
import re

In [ ]:
#ManaBox file
file_path = '/content/ManaBox_Collection.csv'
manabox_df = pd.read_csv(file_path)

In [ ]:
#Create a dictionary to replace mana symbols with readable text
mana_symbols = {
    '{W}': 'White',
    '{U}': 'Blue',
    '{B}': 'Black',
    '{R}': 'Red',
    '{G}': 'Green',
    '{C}': 'Colorless',
    '{T}': 'Tap',
    '//': 'or'
}

In [ ]:
#Create a function to make mana and printed text human-readable
def human_readable_text(text):
    #Replace each mana symbol in the dictionary with its text equivalent
    for symbol, readable_text in mana_symbols.items():
        text = text.replace(symbol, readable_text)
    #Remove remaining braces and add spaces between mana parts
    text = re.sub(r'{|}', '', text)
    text = re.sub(r'(?<=[a-zA-Z])(?=[A-Z])', ' ', text)
    return text

In [ ]:
def fetch_card_data(scryfall_id):
    url = f"https://api.scryfall.com/cards/{scryfall_id}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to fetch data for Scryfall ID: {scryfall_id}")
        return {}

    card_data = response.json()

    # handle dual-face / adventure / modal cards
    face_a = {}
    face_b = {}
    if "card_faces" in card_data and card_data["card_faces"]:
        faces = card_data["card_faces"]
        face_a = faces[0]
        if len(faces) > 1:
            face_b = faces[1]

    def safe_join(value):
        if isinstance(value, list):
            return ", ".join(value)
        return value or ""

    # main metadata
    enriched = {
        # base identity
        "Name": card_data.get("name", ""),
        "Oracle ID": card_data.get("oracle_id", ""),
        "Layout": card_data.get("layout", ""),
        "Rarity": card_data.get("rarity", ""),
        "Lang": card_data.get("lang", ""),

        # color & mana
        "Colors": safe_join(card_data.get("colors", [])),
        "Color Identity": safe_join(card_data.get("color_identity", [])),
        "Mana Cost": card_data.get("mana_cost", ""),
        "Mana Value": card_data.get("cmc", ""),
        "Produced Mana": safe_join(card_data.get("produced_mana", [])),

        # stats
        "Power": card_data.get("power", ""),
        "Toughness": card_data.get("toughness", ""),

        # legality / popularity
        "Commander Legal": card_data.get("legalities", {}).get("commander", ""),
        "EDHREC Rank": card_data.get("edhrec_rank", ""),
        "Digital Availability": card_data.get("digital", ""),

        # descriptive text
        "Type": card_data.get("type_line", ""),
        "Keywords": safe_join(card_data.get("keywords", [])),
        "Printed Text": card_data.get("oracle_text", ""),
        "Flavor Text": card_data.get("flavor_text", ""),

        # set data
        "Set Code": card_data.get("set", ""),
        "Set Name": card_data.get("set_name", ""),
        "Collector Number": card_data.get("collector_number", ""),
        "Release Date": card_data.get("released_at", ""),

        # prices
        "Price USD": card_data.get("prices", {}).get("usd", ""),
        "Price USD Foil": card_data.get("prices", {}).get("usd_foil", ""),

        # media
        "Image URL": card_data.get("image_uris", {}).get("normal", ""),
        "Scryfall URI": card_data.get("scryfall_uri", ""),
    }

    # face-specific fields (if any)
    if face_a:
        enriched.update({
            "Face A Name": face_a.get("name", ""),
            "Face A Type": face_a.get("type_line", ""),
            "Face A Mana Cost": face_a.get("mana_cost", ""),
            "Face A Colors": safe_join(face_a.get("colors", [])),
            "Face A Text": face_a.get("oracle_text", ""),
        })
    if face_b:
        enriched.update({
            "Face B Name": face_b.get("name", ""),
            "Face B Type": face_b.get("type_line", ""),
            "Face B Mana Cost": face_b.get("mana_cost", ""),
            "Face B Colors": safe_join(face_b.get("colors", [])),
            "Face B Text": face_b.get("oracle_text", ""),
        })

    return enriched

In [ ]:
#Initialize a list to hold the complete data
collection_data = []

In [ ]:
#Loop through each row in the ManaBox collection and fetch additional data
for _, row in manabox_df.iterrows():
    scryfall_id = row['Scryfall ID']
    card_data = fetch_card_data(scryfall_id)

    #Combine the data from the ManaBox file with the fetched Scryfall data
    card_info = {
        "Quantity": row["Quantity"],
        "Card Name": row["Name"],
        "Set Code": row["Set code"],
        "Card Number": row["Collector number"],
        "Foil": row["Foil"],
        "Rarity": row["Rarity"],
        "Condition": row["Condition"],
        "ManaBox ID": row["ManaBox ID"],
        "Scryfall ID": scryfall_id,
        "Purchase Price": row["Purchase price"],
        #"Binder Name": row["Binder Name"],
        #"Binder Type": row["Binder Type"],
        "Misprint": row["Misprint"],
        "Altered": row["Altered"],
        "Language": row["Language"],
        "Purchase Price Currency": row["Purchase price currency"]
    }

    #Add Scryfall data
    card_info.update(card_data)
    collection_data.append(card_info)

    #Wait to avoid hitting rate limits
    time.sleep(0.1)

KeyboardInterrupt: 

In [ ]:
#Create a DataFrame from the collected data
collection_df = pd.DataFrame(collection_data)

In [ ]:
#Save the collection as a new Excel file
output_file = '/content/Complete_ManaBox_Collection.xlsx'
collection_df.to_excel(output_file, index=False, engine='openpyxl')

print(f"Collection saved to {output_file}")